In [1]:
import os
from glob import glob
import matplotlib.pyplot as plt
import cv2
import numpy as np
import h5py
from ipywidgets import interact
import pandas as pd
import sys

In [2]:
def gen_list_of_image_names(abs_path, extension):
    list_of_image_names = glob(os.path.join(abs_path, '*'+extension))
    return list_of_image_names

In [3]:
def get_img_idx(path):
    idx = int(path.split('/')[-1].split('_')[0])
    return idx

In [4]:
def gen_dateset(list_of_image_names, square=False, newsize=None, color=True):
    dataset = []
    for img_name in list_of_image_names:
        if color:
            img = cv2.imread(img_name,1) #array 720x1280 rgb
        else:
            img = cv2.imread(img_name,0) #array 720x1280 grayscale
        if (img is None):
            print(img_name,'eh uma imagem vazia')
        else:
            if square:
                border = int((img.shape[1] - img.shape[0])/2)
                frame = frame[:, border : (img.shape[1]-border), :]
            if newsize:
                img = cv2.resize(img, newsize) 
            dataset.append(img)
    #print(type(img))
#     print(len(dataset))
#     print(type(dataset))
    dataset = np.stack(dataset)
    return dataset

In [5]:
def permutate(ds, labels=None):
    p = np.random.permutation(ds.shape[0])
    ds = ds[p]
    if (labels is not None):
        labels = labels[p]
        return ds, labels
    return ds

### importando os targets e pegando os paths dos frames

In [19]:
framespath = '/Users/henriquegoncalves/Documents/UFRJ/PF2/Images/imagens_PL0138/'
classes = ['dano', 'variacao']

cltargets = pd.read_csv(os.path.join(framespath,'targets.csv'),index_col=0)[classes]
imgpaths = gen_list_of_image_names(framespath,'.jpg')
imgpaths.sort(key=lambda path : get_img_idx(path)) #sort by image index
imgpaths = pd.Series(imgpaths)
assert(len(cltargets) == len(imgpaths))

In [20]:
test_video_name = 'HD_C_pl0138_VOL006'
# valid_video_name = 'HD_C_pl0138_VOL007'

te_idx = []
# tr_idx = []
# v_idx = []

# for i in range(len(imgnames)):
te_idx = [i for i,path in enumerate(imgpaths) if test_video_name in path]
# v_idx = [i for i,path in enumerate(imgpaths) if valid_video_name in path]

# tr_idx = [path for path in imgpaths if test_video_name not in path]
# tr_idx = [i for i,path in enumerate(tr_idx) if valid_video_name not in path]

# print(len(tr_idx),len(te_idx),len(v_idx))

In [21]:
cltargets['path'] = imgpaths

In [22]:
# v_df = cltargets.loc[v_idx]
# tr_df = cltargets.loc[tr_idx]
te_df = cltargets.loc[te_idx]

# v_p_df = v_df[v_df[classes].sum(axis=1) > 0]
# v_n_df = v_df[v_df[classes].sum(axis=1) == 0]

te_p_df = te_df[te_df[classes].sum(axis=1) > 0]
te_n_df = te_df[te_df[classes].sum(axis=1) == 0]

# tr_p_df = tr_df[tr_df[classes].sum(axis=1) > 0]
# tr_n_df = tr_df[tr_df[classes].sum(axis=1) == 0]

#cltargets.loc[(cltargets[cl] == 1) | (cltargets.variacao == 1)]

In [23]:
# print(len(v_p_df),len(v_n_df),len(te_p_df),len(te_n_df),len(tr_p_df),len(tr_n_df))

In [24]:
# m_v = min(len(v_p_df),len(v_n_df))
# m_tr = min(len(tr_p_df),len(tr_n_df))
# m_te = min(len(te_p_df),len(te_n_df))

# v_p_df = v_p_df.iloc[np.random.permutation(len(v_p_df))]
# v_n_df = v_n_df.iloc[np.random.permutation(len(v_n_df))]
# tr_p_df = tr_p_df.iloc[np.random.permutation(len(tr_p_df))]
# tr_n_df = tr_n_df.iloc[np.random.permutation(len(tr_n_df))]
# te_p_df = te_p_df.iloc[np.random.permutation(len(te_p_df))]
# te_n_df = te_n_df.iloc[np.random.permutation(len(te_n_df))]

# v_p_df = v_p_df.iloc[0:m_v]
# v_n_df = v_n_df.iloc[0:m_v]

# tr_p_df = tr_p_df.iloc[0:m_tr]
# tr_n_df = tr_n_df.iloc[0:m_tr]

# te_p_df = te_p_df.iloc[0:m_te]
# te_n_df = te_n_df.iloc[0:m_te]

In [25]:
# v_dff = v_p_df.append(v_n_df)
# tr_dff = tr_p_df.append(tr_n_df)
te_dff = te_p_df.append(te_n_df)

# tr_dff = tr_dff.iloc[np.random.permutation(len(tr_dff))]
# v_dff = v_dff.iloc[np.random.permutation(len(v_dff))]
# te_dff = te_dff.iloc[np.random.permutation(len(te_dff))]

In [26]:
# print(len(v_dff),len(tr_dff),len(te_dff))

In [27]:
%%time

X_te = gen_dateset(te_dff.path.values.tolist(), newsize=(224,224))
Y_te = te_dff[classes].values
print('Tamaninho de X_te:',round(sys.getsizeof(X_te)/1e6,2),'MB')

# X_tr = gen_dateset(tr_dff.path.values.tolist(), newsize=(224,224))
# Y_tr = tr_dff[classes].values
# print('Tamaninho de X_tr:',round(sys.getsizeof(X_tr)/1e6,1),'MB')

# X_v = gen_dateset(v_dff.path.values.tolist(), newsize=(224,224))
# Y_v = v_dff[classes].values
# print('Tamaninho de X_v:',round(sys.getsizeof(X_v)/1e6,2),'MB')

Tamaninho de X_te: 327.25 MB
CPU times: user 23.3 s, sys: 1.69 s, total: 25 s
Wall time: 27 s


In [28]:
xgroup = X_te
ygroup = Y_te

def interactive_print(index):
    plt.figure(figsize=(6,6))
    plt.imshow(xgroup[index,:,:,:])
    print(pd.DataFrame([ygroup[index,:]],columns=classes).to_string(index=False))

In [29]:
interact(interactive_print, index = (0,len(xgroup)))

interactive(children=(IntSlider(value=1087, description='index', max=2174), Output()), _dom_classes=('widget-i…

<function __main__.interactive_print(index)>

In [30]:
path = '../datasets/'
filename = 'VaDa_te_224color.h5'
f = os.path.join(path, filename)

h5f = h5py.File(f, 'w')
# h5f.create_dataset('X_tr', data=X_tr)
# h5f.create_dataset('Y_tr', data=Y_tr)
# h5f.create_dataset('X_v', data=X_v)
# h5f.create_dataset('Y_v', data=Y_v)
h5f.create_dataset('X_te', data=X_te)
h5f.create_dataset('Y_te', data=Y_te)
h5f.create_dataset('classes', data=[c.encode() for c in classes])
h5f.close()

NameError: name 'Y_v' is not defined

In [60]:
print(pd.DataFrame([[0,1],[1,0]],columns=classes,index=['sfjnv','cvvsd']))#.to_string(index=False))

       dano  variacao
sfjnv     0         1
cvvsd     1         0


In [32]:
del X_te, Y_te

NameError: name 'X_te' is not defined